In [27]:
from acme import esi_cluster_setup, cluster_cleanup, ParallelMap

import h5py

from nilearn_connectome import compute_connectome

In [10]:
myClient = esi_cluster_setup(n_workers=10, partition="8GBXS")

/gs/home/fuertingers/.conda/envs/rse23/lib/python3.9/site-packages/distributed/node.py:179: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38355 instead
  warnings.warn(


<slurm_cluster_setup> Requested worker-count 10 exceeds `n_workers_startup`: waiting for 1 workers to come online, then proceed


<slurm_cluster_setup> SLURM workers ready: 3/None 	[elapsed time 00:04 | timeout at 01:00]

<slurm_cluster_setup> Cluster dashboard accessible at http://10.100.32.17:38355/status


In [13]:
with ParallelMap(compute_connectome, range(5), result_shape=(264, 264, None)) as pmap:
    results = pmap.compute()

<ParallelMap> INFO: This is ACME v. 2022.12
<ParallelMap> INFO: Attaching to global parallel computing client <Client: 'tcp://10.100.32.17:34461' processes=10 threads=10, memory=74.50 GiB>
<ParallelMap> INFO: Preparing 5 parallel calls of `compute_connectome` using 10 workers
<ParallelMap> INFO: Log information available at /cs/slurm/fuertingers/fuertingers_20230206-160916
100% |██████████| 5/5 [00:40<00:00]
<ParallelMap> INFO: SUCCESS! Finished parallel computation. Results have been saved to /cs/home/fuertingers/ACME_20230206-161156-164963/compute_connectome.h5 with links to data payload located in /cs/home/fuertingers/ACME_20230206-161156-164963/compute_connectome_payload


In [14]:
results

['/cs/home/fuertingers/ACME_20230206-161156-164963/compute_connectome_payload/compute_connectome_0.h5',
 '/cs/home/fuertingers/ACME_20230206-161156-164963/compute_connectome_payload/compute_connectome_1.h5',
 '/cs/home/fuertingers/ACME_20230206-161156-164963/compute_connectome_payload/compute_connectome_2.h5',
 '/cs/home/fuertingers/ACME_20230206-161156-164963/compute_connectome_payload/compute_connectome_3.h5',
 '/cs/home/fuertingers/ACME_20230206-161156-164963/compute_connectome_payload/compute_connectome_4.h5']

In [15]:
pmap.results_container

'/cs/home/fuertingers/ACME_20230206-161156-164963/compute_connectome.h5'

In [24]:
res = h5py.File(pmap.results_container, "r")
print(res["result_0"])

<HDF5 dataset "result_0": shape (264, 264, 5), type "<f8">


In [25]:
res["result_0"][:, :, 0] # First subject

array([[ 1.        ,  0.09640513,  0.06990924, ...,  0.12990164,
        -0.04038437,  0.07309743],
       [ 0.09640513,  1.        ,  0.11786283, ..., -0.02285714,
        -0.06695917,  0.01946163],
       [ 0.06990924,  0.11786283,  1.        , ..., -0.18113036,
        -0.08903197,  0.06609709],
       ...,
       [ 0.12990164, -0.02285714, -0.18113036, ...,  1.        ,
         0.10743258,  0.04901158],
       [-0.04038437, -0.06695917, -0.08903197, ...,  0.10743258,
         1.        ,  0.17943694],
       [ 0.07309743,  0.01946163,  0.06609709, ...,  0.04901158,
         0.17943694,  1.        ]])

In [28]:
cluster_cleanup()

<ParallelMap> INFO: <cluster_cleanup> Successfully shut down cluster fuertingers_20230206-160916 containing 10 workers
